In [ ]:
# Import libraries
import csv
import sys, os
import sqlite3
import calmap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from datetime import timedelta
from collections import deque

In [ ]:
# Read data from the SQLite database
db_connection = sqlite3.connect('../data/fitbit.db')
data = pd.read_sql("select * from daily_summary",db_connection)

In [ ]:
# Data wrangling

#Remove empty entrie
data = data[data['Steps'].notnull()]

# Convert data column to date type
data['Date'] = pd.to_datetime(data['Date'])
# Create weeknumber
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Week'] = data['Date'].dt.week
data['Weekday'] = data['Date'].dt.weekday
data['Day'] = data['Date'].dt.day
data['YearMonth'] = (data['Year'] % 100) * 100 + data['Month']
data['YearWeek'] = (data['Year'] % 100) * 100 + data['Week']

# Percentage of awake time to time in bed (related to efficiency)
data['Awake Percentage'] = data['Sleep Minutes Awake']/data['Time in Bed']*100 

# Sleep start hour. 
dt_format = "%Y-%m-%dT%H:%M:%S.%f"
data['Sleep Start Hour'] = data['Sleep Start Time'].map(lambda x: (datetime.strptime(str(x),dt_format)).hour+(datetime.strptime(str(x),dt_format)).minute/60.0, na_action = 'ignore')
# If the hour is after noon, then subtract 12 so that midnight becomes the 0 reference.

# Midnight is the baseline so that hours can be either + or - from midnight
ind = data[data['Sleep Start Hour'] > 12.0].index.tolist()
data.loc[ind,'Sleep Start Hour'] = data['Sleep Start Hour'].iloc[ind] - 24.0

#Waking up time
data['Wake Up Hour'] = data['Sleep Start Hour']+data['Time in Bed']/60

data.head(5)

In [ ]:
st_mon = data['Steps'].groupby(data['YearMonth']).mean()

fig,axes = plt.subplots(figsize = (18,12), nrows = 2, ncols = 1)

plt.sca(axes[0])
d = data.boxplot(column='Steps', by='YearMonth', ax =axes[0])
d = plt.xticks(np.linspace(1,18,18))

plt.sca(axes[1])
st_mon.plot(kind = 'bar', alpha = 0.5)
r = plt.title('Median step count over the months')

In [ ]:
st_week = data.groupby(['Year', 'Week'])['Steps'].mean()
st_week.plot(kind = 'bar', alpha = 0.5, figsize = (18,5))
r = plt.title('Median step count over the weeks')

In [ ]:
st_week_2017 = data[data['Year'] == 2017].groupby(['Week'])['Steps'].sum()
st_week_2018 = data[data['Year'] == 2018].groupby(['Week'])['Steps'].sum()
st_week_2019 = data[data['Year'] == 2019].groupby(['Week'])['Steps'].sum()

pd_2017 = pd.DataFrame(st_week_2017)
pd_2017.columns = ['2017']
pd_2018 = pd.DataFrame(st_week_2018)
pd_2018.columns = ['2018']
pd_2019 = pd.DataFrame(st_week_2019)
pd_2019.columns = ['2019']
ax = pd_2017.plot(figsize = (18,8))
pd_2018.plot(ax=ax)
pd_2019.plot(ax=ax)
ax.set_title('Steps per week')
r = ax.set_ylabel('Total steps')

In [ ]:
# Read step data per minute
steps_1min = pd.read_sql("select * from Steps_1m",db_connection)

In [ ]:
# create hourly figures
steps_1min['DateTime'] = pd.to_datetime(steps_1min['Date'] + ' ' + steps_1min['Time'], format='%Y-%m-%d %H:%M:%S')
steps_1min = pd.DataFrame(steps_1min)
steps_1min.index = pd.DatetimeIndex(steps_1min['DateTime'])
steps_1u = pd.DataFrame()
steps_1u['steps'] = steps_1min.Steps.resample('H').sum()

In [ ]:
steps_1u['Hour'] = steps_1u.index.hour
hour_steps = steps_1u.groupby([steps_1u.Hour]).mean()
r = hour_steps.plot.bar()

In [ ]:
# Heatmap steps per day
daily_steps = pd.DataFrame()
daily_steps['Date'] = data['Date']
daily_steps['Steps'] = data['Steps']

series = daily_steps
series['Date'] = pd.to_datetime(series['Date'], format='%Y-%m-%d')
series = series.set_index('Date')
series= pd.Series(series['Steps'], index = series.index)
r = calmap.calendarplot(series, monthticks=1, daylabels='MTWTFSS',
                    dayticks=1, linewidth=0,
                    fig_kws=dict(figsize=(16, 6)))

In [ ]:
# Steps per day of the week
weekday_steps = data.groupby(['Weekday']).mean()['Steps']
weekday_steps_tot = data.groupby(['Weekday']).sum()['Steps']
day_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

fig,axes = plt.subplots(figsize = (16,6),nrows = 1, ncols = 2)

plt.sca(axes[0])
ax = axes[0]
ax.bar(weekday_steps.index, weekday_steps)
plt.xticks(weekday_steps.index, day_labels)
plt.ylim(bottom=8000)
plt.title('Mean steps per day of the week')

plt.sca(axes[1])
ax = axes[1]
ax.bar(weekday_steps_tot.index, weekday_steps_tot)
plt.xticks(weekday_steps_tot.index, day_labels)
plt.ylim(bottom=700000)
r= plt.title('Total steps per day of the week')

In [ ]:
# Show days with most and least steps taken

largest = daily_steps.nlargest(5, 'Steps')
smallest = daily_steps[daily_steps['Steps'] > 1000]
smallest = smallest.nsmallest(5, 'Steps', keep='last')

fig,axes = plt.subplots(figsize = (16,6),nrows = 1, ncols = 2)
plt.sca(axes[0])
ax = axes[0]
ax.barh([1,2,3,4,5], largest.Steps)
dates = largest.Date.tolist()
dates = [datetime.strftime(i, "%Y-%m-%d") for i in dates]
plt.yticks([1,2,3,4,5], dates)
plt.xlim(left=10000, right=60000)
plt.title('Days with most steps')
plt.gca().invert_yaxis()
for i, v in enumerate(largest.Steps):
    ax.text(v, i + 1, str(v), color='blue', fontweight='bold')
    
plt.sca(axes[1])
ax = axes[1]
ax.barh([1,2,3,4,5], smallest.Steps)
dates = smallest.Date.tolist()
dates = [datetime.strftime(i, "%Y-%m-%d") for i in dates]
plt.yticks([1,2,3,4,5], dates)
plt.xlim(left=0, right=3500)
plt.title('Days with least steps')
plt.gca().invert_yaxis()
for i, v in enumerate(smallest.Steps):
    ax.text(v, i + 1, str(v), color='blue', fontweight='bold')


In [ ]:
# Read heart rate data per minute
hr_1min = pd.read_sql("select * from Heartrate",db_connection)
# create hourly figures
hr_1min['DateTime'] = pd.to_datetime(hr_1min['Date'] + ' ' + hr_1min['Time'], format='%Y-%m-%d %H:%M:%S')
hr_1min = pd.DataFrame(hr_1min)
hr_1min.index = pd.DatetimeIndex(hr_1min['DateTime'])
hr_1u = pd.DataFrame()
hr_1u['heartrate'] = hr_1min['Heart Rate'].resample('H').mean()
hr_1u['Hour'] = hr_1u.index.hour

In [ ]:
# Hourly heart rate data
hour_hr = hr_1u.groupby([hr_1u.Hour]).mean()
plt.figure(figsize=(16,6))
plt.bar(hour_hr.index, hour_hr.heartrate)
plt.ylim(bottom=40)
plt.title("Heart rate per hour")
r = plt.xticks(hour_hr.index)

In [ ]:
hr_steps_1u = hr_1u.join(steps_1u, lsuffix='_hr', rsuffix='_steps')
hr_steps_1u = hr_steps_1u[hr_steps_1u['steps'] > 1000]

plt.figure(figsize=(16,6))
plt.scatter(hr_steps_1u.steps, hr_steps_1u.heartrate)
plt.title("Heartrate vs Steps (hourly)")
plt.xlabel('Steps')
plt.ylabel('Heart Rate')